## This notebook identifies **characteristic peaks** for **carnitine compounds**
(original notebook content: v_boxplot_carnitines_M+H_for_manuscript.ipynb)

---


- Carnitines were determined by annotation described in **annotation_search_identify_carnitine.ipynb**
- **output is a boxplot (and corresponding dataframes)** describing the characteristic peaks and their stats

---

### About the process
The process for determining consistent patterns of fragmentation is as follows:

1. MS/MS peaks are grouped in 0.01 M/Z bins to understand fragmentation patterns with higher peak precision
2. Peaks attributed to noise are removed by isolating peaks above a normalized intensity of 0.05
3. Minimum percent occurrence for peaks of interest (after removing noise) was specified to occur in at least 20% of the grouped spectra

Final output summarizes the MS/MS peak intensity distribution represented by those peaks occurring at least 20% of the time.

---
## Notebook organization

### Section 1: Reading input data
- MS/MS peak data
- carnitines (identified by annotation)-specific metadata in GNPS Library

### Section 2: Matching peak data with metadata

### Section 3: Investigate peak fragmentation
- Make 0.01 M/Z bins
- Define minimum intensity and percent occurrence parameters
- Identify peaks that satisfy parameters

### Section 4: Plotting

## Input files needed for the Notebook
1. MS/MS peak data
2. Dataframe output of carnitine metadata in GNPS Library from **annotation_search_identify_carnitine.ipynb**

In [1]:
import pandas as pd
import plotly
import plotly.express as px

### Section 1: Read input data

#### Read peak data
- from v_get_peaks_files.ipynb

(Neededed to break into 5 parts due to file size)

In [2]:
all_file_peaks_part_1 = pd.read_parquet('/home/jovyan/work/notebooks/outputs/all_file_peaks_part_1.gzip')

In [3]:
all_file_peaks_part_2 = pd.read_parquet('/home/jovyan/work/notebooks/outputs/all_file_peaks_part_2.gzip')

In [4]:
all_file_peaks_part_3 = pd.read_parquet('/home/jovyan/work/notebooks/outputs/all_file_peaks_part_3.gzip')

In [5]:
all_file_peaks_part_4 = pd.read_parquet('/home/jovyan/work/notebooks/outputs/all_file_peaks_part_4.gzip')

In [6]:
all_file_peaks_part_5 = pd.read_parquet('/home/jovyan/work/notebooks/outputs/all_file_peaks_part_5.gzip')

#### Read carnitine metadata from GNPS Library
- from annotation_search_identify_carnitine.ipynb

In [7]:
carnitine_table = pd.read_csv('/home/jovyan/work/notebooks/outputs/library_df_carnitine_case_insen_M+H.csv',sep=',', index_col='spectrum_id')

In [8]:
# list of spectrum_id
carnitine_table_ID = carnitine_table.index.to_list()

### Section 2: Matching peak data with metadata
- Need to identify MS/MS peak data for spectra identified as carnitines in GNPS Library by annotation search

In [9]:
all_file_peaks_part_1_carnitine = all_file_peaks_part_1[all_file_peaks_part_1.index.isin(carnitine_table_ID)]

In [10]:
all_file_peaks_part_2_carnitine = all_file_peaks_part_2[all_file_peaks_part_2.index.isin(carnitine_table_ID)]

In [11]:
all_file_peaks_part_3_carnitine = all_file_peaks_part_3[all_file_peaks_part_3.index.isin(carnitine_table_ID)]

In [12]:
all_file_peaks_part_4_carnitine = all_file_peaks_part_4[all_file_peaks_part_4.index.isin(carnitine_table_ID)]

In [13]:
all_file_peaks_part_5_carnitine = all_file_peaks_part_5[all_file_peaks_part_5.index.isin(carnitine_table_ID)]

In [14]:
# Combine individual dataframes to make complete dataframe of all peak data associated with carnitines
all_file_peaks_carnitine = pd.concat([all_file_peaks_part_1_carnitine, all_file_peaks_part_2_carnitine, all_file_peaks_part_3_carnitine,
                                      all_file_peaks_part_4_carnitine, all_file_peaks_part_5_carnitine], axis=0)

In [15]:
all_file_peaks_carnitine

,level_0,index,i,i_norm,i_tic_norm,mz,mz_nl,precmz
scan,,,,,,,,
CCMSLIB00000216197,399442,0,2177.529053,0.056863,0.034625,73.961777,126.118223,200.080
CCMSLIB00000216197,399443,1,25.951593,0.000678,0.000413,74.977188,125.102812,200.080
CCMSLIB00000216197,399444,2,503.461060,0.013147,0.008006,84.048637,116.031363,200.080
CCMSLIB00000216197,399445,3,21725.027344,0.567317,0.345450,102.003937,98.076063,200.080
CCMSLIB00000216197,399446,4,37.872719,0.000989,0.000602,158.435135,41.644865,200.080
...,...,...,...,...,...,...,...,...
CCMSLIB00000221337,35834,3,473.000000,0.473473,0.229500,103.041199,59.071801,162.113
CCMSLIB00000221337,35835,4,999.000000,1.000000,0.484716,162.113007,-0.000007,162.113
CCMSLIB00000221529,36687,0,999.000000,1.000000,0.676371,85.029900,119.094100,204.124


In [16]:
# check that all spectrum_id are accounted for
len(all_file_peaks_carnitine.index.unique().to_list())

387

### Section 3: Investigate peak fragmentation

In [17]:
# rename combined dataframe
peak_df = all_file_peaks_carnitine

In [18]:
# Identifying small bins of MZ values
peak_df['mz_binned_small'] = peak_df['mz'].round(decimals = 2)
unique_mz_binned = peak_df['mz_binned_small'].unique()

In [19]:
peak_df.head(5)

,level_0,index,i,i_norm,i_tic_norm,mz,mz_nl,precmz,mz_binned_small
scan,,,,,,,,,
CCMSLIB00000216197,399442,0,2177.529053,0.056863,0.034625,73.961777,126.118223,200.08,73.96
CCMSLIB00000216197,399443,1,25.951593,0.000678,0.000413,74.977188,125.102812,200.08,74.98
CCMSLIB00000216197,399444,2,503.461060,0.013147,0.008006,84.048637,116.031363,200.08,84.05
CCMSLIB00000216197,399445,3,21725.027344,0.567317,0.345450,102.003937,98.076063,200.08,102.00
CCMSLIB00000216197,399446,4,37.872719,0.000989,0.000602,158.435135,41.644865,200.08,158.44


In [20]:
# Defining parameters --> can be modified by user

intensitynormmin  = 0.05
percentoccurmin = 20

In [21]:
# identifying peaks that satisfy minimum normalized intensity parameter

filtered_peak_df_i_norm = peak_df[peak_df["i_norm"] >= intensitynormmin]

In [22]:
# For counting percent occurrence of peaks above miniumum intensity
occurs_above_intensitynormmin = {}

# Total number of spectral IDs
total_ids = len(peak_df.index.unique())

for peak in unique_mz_binned:
    mz_df_above_intensitynormmin = filtered_peak_df_i_norm.loc[(filtered_peak_df_i_norm['mz_binned_small'] == peak)]

    # Number of spectra where peak occurs above miniumum intensity
    peak_occurs_above_intensitynormmin = len(mz_df_above_intensitynormmin)

    if peak_occurs_above_intensitynormmin/total_ids >= (percentoccurmin/100):
        occurs_above_intensitynormmin[peak] = peak_occurs_above_intensitynormmin/total_ids

In [23]:
# Filtering to only include peaks that are present in at least 20% of the scans
filtered_peak_df = filtered_peak_df_i_norm[filtered_peak_df_i_norm["mz_binned_small"].isin(occurs_above_intensitynormmin.keys())]

In [24]:
filtered_peak_df

,level_0,index,i,i_norm,i_tic_norm,mz,mz_nl,precmz,mz_binned_small
scan,,,,,,,,,
CCMSLIB00000577974,8496,3,5.706804e+06,0.284907,0.151077,60.081528,86.038472,146.120,60.08
CCMSLIB00000577982,9113,6,5.514857e+06,0.107815,0.081476,60.081532,102.028468,162.110,60.08
CCMSLIB00000577990,9792,13,3.082949e+07,0.105989,0.077327,60.081486,284.198514,344.280,60.08
CCMSLIB00000577990,9806,27,2.908738e+08,1.000000,0.729578,85.028687,259.251313,344.280,85.03
CCMSLIB00003134527,5818,3,5.757238e+05,1.000000,0.561997,85.028221,259.250779,344.279,85.03
...,...,...,...,...,...,...,...,...,...
CCMSLIB00000221015,34821,3,3.950000e+02,0.395395,0.124606,60.081902,102.031098,162.113,60.08
CCMSLIB00000221015,34823,5,9.990000e+02,1.000000,0.315142,85.028702,77.084298,162.113,85.03
CCMSLIB00000221337,35831,0,1.320000e+02,0.132132,0.064047,60.082901,102.030099,162.113,60.08


In [25]:
# reshaping and renaming
peak_ratio_df = pd.DataFrame.from_dict(occurs_above_intensitynormmin, orient='index')
peak_ratio_df.index.name = 'mz_binned_small'
peak_ratio_df = peak_ratio_df.rename(columns={0: "percent_occurrence"})

In [26]:
# visualize peaks by percent occurrence
peak_ratio_df

,percent_occurrence
mz_binned_small,
60.08,0.475452
85.03,0.581395


### Section 4: Plotting

In [ ]:
with open('/home/jovyan/work/notebooks/outputs/output_box_peak_carnitine_M+H_df.html', 'w') as f:

        ax = px.box(filtered_peak_df, x='mz_binned_small',y = 'i_norm')

        # Use hidden line below to graph ratio of MISSING peaks
        #ax.add_bar(x=peak_ratio_df.index, y=-peak_ratio_df["ratio of missing peaks"], name = "ratio of spectra with missing peaks")
        
        ax.add_bar(x=peak_ratio_df.index, y=-peak_ratio_df["percent_occurrence"], name = "ratio of spectra where at least " + str(percentoccurmin)+"% contain peak")
        
        ax.update_layout(title_text="MS/MS Peak Intensity Distribution for " +"Carnitines")
        
        f.write(ax.to_html(full_html=False, include_plotlyjs='cdn'))

### Save files

In [30]:
filtered_peak_df.reset_index().to_csv(
    '/home/jovyan/work/notebooks/carnitine_notebooks/files_for_boxplot/filtered_peak_df_carnitines_M+H.csv', sep=',', index=False)

In [33]:
peak_ratio_df.reset_index().to_csv(
    '/home/jovyan/work/notebooks/carnitine_notebooks/files_for_boxplot/peak_ratio_df_carnitines_M+H.csv', sep=',', index=False)